# Uitleg over de functie

# Setup

In [54]:
import os
import json
import random
from pyowm import OWM
from pyowm.caches.lrucache import LRUCache
import datetime
import math
import subprocess
from statistics import mean

import sys
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
sp = spotipy.Spotify()

API_key = '6ff7fde73bb9cccf854fd26d31b4fe6d'
owm = OWM(API_key)
cache = LRUCache()



GLOBALMusicDataframe = pd.DataFrame()
GLOBALCurrentSongList = pd.DataFrame()

In [51]:
def get_playlist_tracks(credentials,username,playlist_id):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #get tracklisting from playlist data
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    #ensure collection of data after more than 100 requests
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_features(tracks):
    #set scope to retreive public data
    scope_playlist = 'playlist-modify-public'
    #provide token using given credentials
    #credentials are: username, public key, private key and redirect_uri
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make empty list to collect dictionaries for dataframe
    dataframe = []
    for x in tracks:
        try:
            track = x['track']
            #get audio features per track
            feat = (sp.audio_features(tracks=[track['id']])[0])
            #get track and artist info
            art_dict = {'artist':track['artists'][0]['name'], 'title':track['name'], 'popularity':track['popularity']}
            #combine both dictionaries
            art_dict.update(feat)
            #make dataframe from dictionary
            dataframe.append(art_dict)
        except:
            print("Oh no!",sys.exc_info()[0],"happend")
    #filter dataframe to neccesary columns
    df = pd.DataFrame(dataframe)
    df = df.filter(['title', 'artist', 'danceability','energy', 'loudness', 'tempo','valence', 'popularity','id'])
    df = df.set_index('title')
    print("Finished collecting Songs")
    return df

def add_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #make selection based on feature input
    dataframe = dataframe.drop_duplicates()
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False))] 
    #remove unwanted artists
    # & (~dataframe['artist'].isin(['Ariana Grande']))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    if len(songs) > 0:
        scope_playlist = 'playlist-modify-public'
        token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
        sp = spotipy.Spotify(auth=token)
        sp.user_playlist_add_tracks(credentials[0],playlist, songs[:100])
        print("Songs Succesfully Added to Playlist")
    else:
        print("There are no suitable songs in your collection")
        
    global GLOBALCurrentSongList
    GLOBALCurrentSongList = selection['artist']
    
def remove_tracks_from_df(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    dataframe = dataframe.drop_duplicates()
    selection = dataframe.loc[(dataframe['danceability'].between(features[0]-0.1, features[0]+0.1, inclusive=False)) &\
    (dataframe['energy'].between(features[1]-0.2, features[1]+0.2, inclusive=False)) &\
    (dataframe['loudness'].between(features[2]-5, features[2]+5, inclusive=False)) &\
    (dataframe['tempo'].between(features[3]-40, features[3]+40, inclusive=False)) &\
    (dataframe['valence'].between(features[4]-0.5, features[4]+0.5, inclusive=False))]
    selection.sort_values(by=['popularity'], ascending=False)
    songs = selection['id'].tolist()
    
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    sp.user_playlist_remove_all_occurrences_of_tracks(credentials[0],playlist, songs[:100])
    print("Songs Succesfully Removed from Playlist")
    
def update_store_list(credentials,playlist,dataframe,features):
    scope_playlist = 'playlist-modify-public'
    token = util.prompt_for_user_token(credentials[0],scope_playlist,credentials[1],credentials[2],credentials[3])
    sp = spotipy.Spotify(auth=token)
    #store_list = get_playlist_tracks(credentials,'zwamborn','4jwcDPU0HyEw0cOAPPFOdp')
    store_list = get_playlist_tracks(credentials,'bartw26396','5yJfsUa3aWq20QhPLGwtig')
    if len(store_list) == 0:
        print('Updating Playlist...')
        add_tracks_from_df(credentials,playlist,dataframe,features)
    else:
        songs =[]
        for x in store_list:
            songs.append(x['track']['id'])
        print('Clearing Playlist...')
        sp.user_playlist_remove_all_occurrences_of_tracks(credentials[0],playlist, songs[:100])
        print("Songs Succesfully Removed from Playlist")
        print('Updating Playlist...')
        add_tracks_from_df(credentials,playlist,dataframe,features)

In [35]:
credentials = ['bartw26396', '5711bc132b4c48ceb5bbd19cd65b1e63', 'f507991961c948d8bf1b62ae6ef5ab15', 'http://localhost']
#credentials = ['zwamborn', '5711bc132b4c48ceb5bbd19cd65b1e63', 'f507991961c948d8bf1b62ae6ef5ab15', 'http://localhost']
playlists = {'Kasper Langendoen':'4W7jnrqeKfVEnb1BVHMG5b', 'Top 50 Wereld':'37i9dQZEVXbMDoHDwVN2tF', 'NPO Radio 2':'1DTzz7Nh2rJBnyFbjsH1Mh',\
'daryl zandvliet':'6PoHyrIELxnRlRKOsI5yhW', 'Slam Official':'0OdWlUFdB6Lio5dIdXY81O', 'Bouke Bosma':'70aT8IllF7t6CLcPf2pt99'}


def BuildMusicDataFrame(credentials, playlists) :
    datafroem = pd.DataFrame()
    for key, value in playlists.items():
        tracklist = get_playlist_tracks(credentials, key,value)
        data = get_features(tracklist)
        datafroem = datafroem.append(data)
    global GLOBALMusicDataframe
    GLOBALMusicDataframe = datafroem
    
    return datafroem

In [4]:
#ranges

decibel_low= range(0,50)
decibel_medium= range(50,70)
decibel_noise= range(70,90)
decibel_loud= range(90,151)

decibel_list= [("decibel_low",decibel_low), ("decibel_medium",decibel_medium), ("decibel_noise",decibel_noise), ("decibel_loud",decibel_loud)]

people_little= range(0,10)
people_medium= range(10,20)
people_busy= range(20,30)
people_crowded= range(30,101)

people_list= [('people_little',people_little), ('people_medium',people_medium), ('people_busy',people_busy), ('people_crowded',people_crowded)]

temperature_freezing= range(-10, 5)
temperature_cold= range(5,15)
temperature_warm= range(15,25)
temperature_hot= range(25,45)

temperature_list = [('temperature_freezing',temperature_freezing), ('temperature_cold',temperature_cold) , ('temperature_warm',temperature_warm), ('temperature_hot',temperature_hot)]

rain_little= range(0,20)
rain_some= range(20,40)
rain_medium= range(40,60)
rain_much= range(60,80)
rain_heavy = range(80,101)

rain_list= [('rain_little',rain_little ), ('rain_some',rain_some), ('rain_medium',rain_medium), ('rain_much',rain_much), ('rain_heavy',rain_heavy)]

clouds_little= range(0,20)
clouds_some= range(20,40)
clouds_medium= range(40,60)
clouds_much= range(60,80)
clouds_only= range(80,101)

clouds_list= [('clouds_little',clouds_little), ('clouds_some',clouds_some), ('clouds_medium',clouds_medium), ('clouds_much',clouds_much), ('clouds_only',clouds_only)]

time_early= range(9,11)
time_beforelunch= range(11,13)
time_afterlunch= range(13,17)
time_closing= range(17,18)

time_list= [('time_early',time_early), ('time_beforelunch',time_beforelunch), ('time_afterlunch',time_afterlunch), ('time_closing',time_closing)]


# Retrieving Surrounding Data

In [5]:
def filebuild():
    if os.path.exists("mac_adresses.json"):
        os.remove("mac_adresses.json")
        #print("Remove")
    else: 
        os.system("howmanypeoplearearound --out mac_adresses.json --adapter en0 --scantime 20 --sort")
        #print("Build")
        
def WifiNum() :
    # https://github.com/schollz/howmanypeoplearearound

    filebuild()

    # This piece of code uses the Wi-Fi adapter to track the amount of active phones around
    #os.system("howmanypeoplearearound --out hoidoeiyooppo.json --adapter en0 --sort")

    with open('mac_adresses.json') as f:
        data = json.load(f)

    all_cellphones = [x for x in data['cellphones']]
    threshold = -67 #Around 10 meters
    cellphones_within_threshold = [x for x in all_cellphones if x['rssi'] > threshold]

    print("NumPeople",len(cellphones_within_threshold))

    #####################
    #Calculates the distance
    #####################
    # Power = waarde op 1 meter afstand

    # Constant depends on the Environmental factor. Range 2-4

    #Power = -47.27906976744186
    #RSSI = -67
    #N = 2

    #Distance = 10**((int(Power)-int(RSSI))/(10 * int(N)))
    #print(Distance)
    filebuild()
    return len(cellphones_within_threshold)
#WifiNum()

In [31]:
def DecibelNum():
    #Install program, if installed this can also be removed from the formula
    os.system('pip install soundmeter') 

    #Collect the sound for a period of time of 5seconds
    batcmd="soundmeter --collect --seconds 5" 

    #Get the terminal result
    result1 = subprocess.check_output(batcmd, shell=True) 

    #Get the average rms from the x seconds measurement out of the terminal string (limited to 120db)
    average_rms = int(result1[-7:-1])

    #Convert the RMS value to Db(a)
    amount_of_db = 20 * math.log10(average_rms)

    #round to two decimal
    amount_of_db_rounded = round(amount_of_db, 2)
    
    return amount_of_db_rounded
DecibelNum()

63.6

In [7]:
def DecibelNum2():
    declist=[]
    for i in range(5):
        declist.append(random.randint(30,130))
    dec = sum(declist)/len(declist)
    return dec

In [8]:
def WeatherNum() :
    place = owm.weather_at_place('Muiden,NL')
    weathercall = place.get_weather()
    cloudiness = weathercall.get_clouds()
    rain_status = weathercall.get_rain()
    if len(rain_status) > 0:
        rkeys = list(rain_status.keys())
        rain_mm = rain_status[rkeys[0]]
    else:
        rain_mm = 0
    temperature = weathercall.get_temperature(unit='celsius')['temp']

    return [cloudiness,rain_mm, temperature]
    
#WeatherNum()

In [32]:
def getNums() :
    numPeople = WifiNum()
    decibel = DecibelNum()
    weather = WeatherNum()
    temperature = weather[2]
    clouds = weather[0]
    rain = weather[1]
    now = datetime.datetime.now()
    time = now.hour
    
    return decibel, numPeople, temperature, rain, clouds, time

#getNums()

# Setting parameters

In [10]:
def Parameter_ranges(decibel, people, temperature, rain, clouds, time):
    ruis_range= str
    mensen_range = str
    temp_range = str
    regen_range = str
    wolk_range = str
    uur_range = str
    
    for ruis in decibel_list:
        for db in ruis[1]:
            if db == decibel:
                ruis_range = ruis[0]
            
    for mensen in people_list:
        for mens in mensen[1]:
            if mens==people:
                mensen_range = mensen[0]
            
    for temp in temperature_list:
        for c in temp[1]:
            if c==temperature:
                temp_range = temp[0]
    
    for regen in rain_list:
        for m in regen[1]:
            if m==rain:
                regen_range = regen[0]
            
    for wolk in clouds_list:
        for w in wolk[1]:
            if w==clouds:
                wolk_range = wolk[0]
    
    for uur in time_list:
        for s in uur[1]:
            if s==time:
                uur_range = uur[0]
    
    return ruis_range, mensen_range, temp_range, regen_range, wolk_range, uur_range

In [11]:
Parameter_ranges(50, 10, 20, 100, 100, 15)

('decibel_medium',
 'people_medium',
 'temperature_warm',
 'rain_heavy',
 'clouds_only',
 'time_afterlunch')

In [12]:
def Danceability(decibel, people, temperature, rain, clouds, time):
    
    parameter_returns= Parameter_ranges(decibel, people, temperature, rain, clouds, time)
    
    score_ruis= 0
    score_mens= 0
    score_temp= 0
    score_regen= 0
    score_wolk= 0 
    score_uur = 0
    score_total = 0
    
    decibel_tuple= [("decibel_low", 9), ("decibel_medium",7), ("decibel_noise",6),("decibel_loud",3)]
    people_tuple= [('people_little',9), ('people_medium',7), ('people_busy',5), ('people_crowded',4)]
    temperature_tuple = [('temperature_freezing',4), ('temperature_cold',6) , ('temperature_warm',8), ('temperature_hot',9)]
    rain_tuple = [('rain_little',9), ('rain_some',7), ('rain_medium',6), ('rain_much',4), ('rain_heavy',3)]
    clouds_tuple = [('clouds_little',9), ('clouds_some',7), ('clouds_medium',6), ('clouds_much',5), ('clouds_only',4)]
    time_tuple = [('time_early',5), ('time_beforelunch',7), ('time_afterlunch',9), ('time_closing',5)]
    
    for x in decibel_tuple:
        if x[0] == parameter_returns[0]:
            score_ruis = x[1]*30
    
    for x in people_tuple:
        if x[0] == parameter_returns[1]:
            score_mens = x[1]*20
            
    for x in temperature_tuple:
        if x[0] == parameter_returns[2]:
            score_temp = x[1]*15
    
    for x in rain_tuple:
        if x[0] == parameter_returns[3]:
            score_regen = x[1]*5
    
    for x in clouds_tuple:
        if x[0] == parameter_returns[4]:
            score_wolk = x[1]*10
    
    for x in time_tuple:
        if x[0] == parameter_returns[5]:
            score_uur = x[1]*20
    
    da_total= (score_ruis + score_mens + score_temp + score_regen + score_wolk + score_uur)/1000
    return da_total



In [13]:
Danceability(80, 35, 35, 0, 0, 14)

0.71

In [14]:
def Energy(decibel, people, temperature, rain, clouds, time):
    
    parameter_returns= Parameter_ranges(decibel, people, temperature, rain, clouds, time)
    
    score_ruis= 0
    score_mens= 0
    score_temp= 0
    score_regen= 0
    score_wolk= 0 
    score_uur = 0
    score_total = 0
    
    decibel_tuple= [("decibel_low", 9), ("decibel_medium",7), ("decibel_noise",4),("decibel_loud",2)]
    people_tuple= [('people_little',8), ('people_medium',7), ('people_busy',6), ('people_crowded',4)]
    temperature_tuple = [('temperature_freezing',5), ('temperature_cold',7) , ('temperature_warm',8), ('temperature_hot',9)]
    rain_tuple = [('rain_little',9), ('rain_some',7), ('rain_medium',5), ('rain_much',4), ('rain_heavy',2)]
    clouds_tuple = [('clouds_little',9), ('clouds_some',8), ('clouds_medium',6), ('clouds_much',5), ('clouds_only',4)]
    time_tuple = [('time_early',4), ('time_beforelunch',6), ('time_afterlunch',9), ('time_closing',3)]
    
    for x in decibel_tuple:
        if x[0] == parameter_returns[0]:
            score_ruis = x[1]*30
    
    for x in people_tuple:
        if x[0] == parameter_returns[1]:
            score_mens = x[1]*20
            
    for x in temperature_tuple:
        if x[0] == parameter_returns[2]:
            score_temp = x[1]*15
    
    for x in rain_tuple:
        if x[0] == parameter_returns[3]:
            score_regen = x[1]*5
    
    for x in clouds_tuple:
        if x[0] == parameter_returns[4]:
            score_wolk = x[1]*10
    
    for x in time_tuple:
        if x[0] == parameter_returns[5]:
            score_uur = x[1]*20
    
    en_total= (score_ruis + score_mens + score_temp + score_regen + score_wolk + score_uur)/1000
    return en_total

In [15]:
Energy(80, 35, 35, 0, 0, 14)

0.65

In [16]:
def Loudness(decibel, people, temperature, rain, clouds, time):
    #note that for this function we use different coefficients, 
    #because not all parameters were considered relevant for predicting loudness.
    
    parameter_returns= Parameter_ranges(decibel, people, temperature, rain, clouds, time)
    
    score_ruis= 0
    score_mens= 0
    score_uur = 0
    score_total = 0
    
    decibel_tuple= [("decibel_low", 9), ("decibel_medium",6), ("decibel_noise",3),("decibel_loud",1)]
    people_tuple= [('people_little',9), ('people_medium',6), ('people_busy',4), ('people_crowded',2)]
    time_tuple = [('time_early',4), ('time_beforelunch',7), ('time_afterlunch',8), ('time_closing',4)]
    
    for x in decibel_tuple:
        if x[0] == parameter_returns[0]:
            score_ruis = x[1]*50
    
    for x in people_tuple:
        if x[0] == parameter_returns[1]:
            score_mens = x[1]*20
    
    for x in time_tuple:
        if x[0] == parameter_returns[5]:
            score_uur = x[1]*30
    
    lo_total= (score_ruis + score_mens + score_uur)/10
    value = ((lo_total * (0 - -30) / 100) + -30)
    #val = ((percent * (max - min) / 100) + min
    return value

In [17]:
Loudness(20, 5, 35, 100, 100, 10)

-7.5

In [18]:
def Tempo(decibel, people, temperature, rain, clouds, time):
    #note that for this function we use different coefficients, 
    #because not all parameters were considered relevant for predicting loudness.
    
    parameter_returns= Parameter_ranges(decibel, people, temperature, rain, clouds, time)
    
    score_ruis= 0
    score_mens= 0
    score_temp= 0
    score_uur = 0
    score_total = 0
    
    decibel_tuple= [("decibel_low", 9), ("decibel_medium",7), ("decibel_noise",5),("decibel_loud",3)]
    people_tuple= [('people_little',8), ('people_medium',7), ('people_busy',4), ('people_crowded',3)]
    temperature_tuple = [('temperature_freezing',5), ('temperature_cold',7) , ('temperature_warm',9), ('temperature_hot',5)]
    time_tuple = [('time_early',4), ('time_beforelunch',9), ('time_afterlunch',6), ('time_closing',4)]
    
    for x in decibel_tuple:
        if x[0] == parameter_returns[0]:
            score_ruis = x[1]*30
    
    for x in people_tuple:
        if x[0] == parameter_returns[1]:
            score_mens = x[1]*20
                               
    for x in temperature_tuple:
        if x[0] == parameter_returns[2]:
            score_temp = x[1]*20
    
    for x in time_tuple:
        if x[0] == parameter_returns[5]:
            score_uur = x[1]*30
    
    lo_total= (score_ruis + score_mens + score_temp+ score_uur)/10
    value = ((lo_total * (200 - 50) / 100) + 50)
    return value

In [19]:
Tempo(20, 5, 10, 100, 100, 10)

153.5

In [20]:
def Valence(decibel, people, temperature, rain, clouds, time):
    
    parameter_returns= Parameter_ranges(decibel, people, temperature, rain, clouds, time)
    
    score_ruis= 0
    score_mens= 0
    score_temp= 0
    score_regen= 0
    score_wolk= 0 
    score_uur = 0
    score_total = 0
    
    decibel_tuple= [("decibel_low", 9), ("decibel_medium",7), ("decibel_noise",5),("decibel_loud",3)]
    people_tuple= [('people_little',9), ('people_medium',7), ('people_busy',5), ('people_crowded',4)]
    temperature_tuple = [('temperature_freezing',3), ('temperature_cold',5) , ('temperature_warm',10), ('temperature_hot',8)]
    rain_tuple = [('rain_little',8), ('rain_some',7), ('rain_medium',5), ('rain_much',4), ('rain_heavy',3)]
    clouds_tuple = [('clouds_little',10), ('clouds_some',8), ('clouds_medium',6), ('clouds_much',4), ('clouds_only',3)]
    time_tuple = [('time_early',4), ('time_beforelunch',9), ('time_afterlunch',7), ('time_closing',5)]
    
    for x in decibel_tuple:
        if x[0] == parameter_returns[0]:
            score_ruis = x[1]*30
    
    for x in people_tuple:
        if x[0] == parameter_returns[1]:
            score_mens = x[1]*20
            
    for x in temperature_tuple:
        if x[0] == parameter_returns[2]:
            score_temp = x[1]*15
    
    for x in rain_tuple:
        if x[0] == parameter_returns[3]:
            score_regen = x[1]*5
    
    for x in clouds_tuple:
        if x[0] == parameter_returns[4]:
            score_wolk = x[1]*10
    
    for x in time_tuple:
        if x[0] == parameter_returns[5]:
            score_uur = x[1]*20
    
    va_total= (score_ruis + score_mens + score_temp + score_regen + score_wolk + score_uur)/1000
    return va_total

In [21]:
Valence(20, 5, 10, 100, 100, 10)

0.65

In [22]:
def RunAll(bron):
    decibel, people, temperature, rain, clouds, time = bron[0], bron[1], bron[2], bron[3], bron[4], bron[5]
    danceability = Danceability(decibel, people, temperature, rain, clouds, time)
    energy = Energy(decibel, people, temperature, rain, clouds, time)
    loudness= Loudness(decibel, people, temperature, rain, clouds, time)
    tempo= Tempo(decibel, people, temperature, rain, clouds, time)
    valence= Valence(decibel, people, temperature, rain, clouds, time)
    
    print(bron)
    print('danceability:', danceability)
    print('energy:', energy)
    print('loudness:', loudness)
    print('tempo:', tempo)
    print('valence:', valence)
    return [danceability, energy, loudness, tempo, valence]

In [23]:
nummers = 50, 15, 10, 40, 80, 10
RunAll(nummers)

(50, 15, 10, 40, 80, 10)
danceability: 0.61
energy: 0.6
loudness: -13.8
tempo: 141.5
valence: 0.56


[0.61, 0.6, -13.8, 141.5, 0.56]

In [24]:
numbers = getNums()

NumPeople 5


In [25]:
def StatRetrieval(source, features):
    namelist1 = ['danceability','energy','loudness','tempo','valence']
    namelist2 = ['Decibel', 'NumPeople', 'Temperature', 'Rain', 'Clouds', 'Time']
    Numlist = []
    DataList = []
    for i in range(len(features)):
        Numlist.append((namelist1[i],features[i]))
    for i in range(len(source)):
        DataList.append((namelist2[i],source[i]))
    
    return Numlist, DataList

def spotifyListBuilder(credits,musicData):
    surroundings = getNums()
    features = RunAll(surroundings)
    update_store_list(credits,'5yJfsUa3aWq20QhPLGwtig',musicData,features)
    print("Done, Spotify Filled",features)
    return surroundings, features


def RunTheCode(new, credentials, playlists):
    if new:
        BuildMusicDataFrame(credentials, playlists)
        print("MusicDataFrame is filled")
    result = spotifyListBuilder(credentials,GLOBALMusicDataframe)
    print("Fully Done!")
    currentStats = StatRetrieval(result[0],result[1])
    
    return currentStats

In [55]:
def Start(Keuze):
    currentStats = RunTheCode(Keuze, credentials, playlists)
    return currentStats, GLOBALCurrentSongList

In [57]:
Start(True)

retrying ...6secs
Finished collecting Songs
Finished collecting Songs
retrying ...3secs
retrying ...6secs
retrying ...2secs
retrying ...5secs
retrying ...6secs
retrying ...3secs
retrying ...3secs
retrying ...3secs
retrying ...4secs
retrying ...5secs
retrying ...5secs
retrying ...4secs
retrying ...5secs
retrying ...4secs
retrying ...3secs
Finished collecting Songs
retrying ...5secs
retrying ...6secs
retrying ...5secs
retrying ...6secs
retrying ...5secs
Finished collecting Songs
Finished collecting Songs
retrying ...5secs
retrying ...5secs
retrying ...6secs
retrying ...5secs
retrying ...6secs
retrying ...6secs
retrying ...5secs
Finished collecting Songs
MusicDataFrame is filled
NumPeople 3
(56.86, 3, 12.68, 0, 100, 15)
danceability: 0.445
energy: 0.425
loudness: -17.4
tempo: 101.0
valence: 0.39
Updating Playlist...
Songs Succesfully Added to Playlist
Done, Spotify Filled [0.445, 0.425, -17.4, 101.0, 0.39]
Fully Done!


(([('danceability', 0.445),
   ('energy', 0.425),
   ('loudness', -17.4),
   ('tempo', 101.0),
   ('valence', 0.39)],
  [('Decibel', 56.86),
   ('NumPeople', 3),
   ('Temperature', 12.68),
   ('Rain', 0),
   ('Clouds', 100),
   ('Time', 15)]),
                                                         artist  danceability  \
 title                                                                          
 Mistakes                                             Arnautica         0.489   
 Avond                                       Boudewijn de Groot         0.456   
 De Boer Is De Keerl                                    Normaal         0.519   
 Wish You Were Here                                  Pink Floyd         0.499   
 Comfortably Numb - 2011 Remastered Version          Pink Floyd         0.472   
 ...                                                        ...           ...   
 Kissing a Fool                                  George Michael         0.394   
 (Everything I Do) I Do It 

In [58]:
GLOBALMusicDataframe

,artist,danceability,energy,loudness,tempo,valence,popularity,id
title,,,,,,,,
Keep Control - ARTBAT Remix,Sono,0.769,0.762,-9.669,123.002,0.119,43,6n8wkmgsU4wCJdM2rky0yb
Eyes On Fire - Michael Bibi Remix Edit,Blue Foundation,0.800,0.683,-8.771,127.995,0.625,47,5kNOW0qMt6XN4d4rQUUu03
Remember Me - Franky Rizardo Radio Edit,Blue Boy,0.835,0.898,-7.126,127.989,0.925,41,5pUvWWkn5yONeE8DWBTrgC
DT64,Moguai,0.743,0.945,-8.660,129.992,0.172,57,5z7VT7KoeFLT1rjeyDzdMI
Deadeye,Sorley,0.810,0.915,-7.618,124.994,0.741,45,5o7M979CiUYnY1M2l6VMQ0
...,...,...,...,...,...,...,...,...
You're The One That I Want - From “Grease”,John Travolta,0.758,0.612,-11.403,106.945,0.822,74,0B9x2BRHqj3Qer7biM3pU3
Mr. Jones,Counting Crows,0.578,0.817,-6.542,141.621,0.726,75,5DiXcVovI0FcY2s0icWWUu
Times Are Changing,DI-RECT,0.501,0.810,-6.106,122.968,0.285,47,2ChLCoNCOIXSKEgrRueGsy


In [56]:
Start(False)

NumPeople 6
(65.47, 6, 12.68, 0, 100, 15)
danceability: 0.445
energy: 0.425
loudness: -17.4
tempo: 101.0
valence: 0.39
Clearing Playlist...
Songs Succesfully Removed from Playlist
Updating Playlist...


KeyError: 'danceability'

In [61]:
GLOBALCurrentSongList['artist']

title
Mistakes                                               Arnautica
Avond                                         Boudewijn de Groot
De Boer Is De Keerl                                      Normaal
Wish You Were Here                                    Pink Floyd
Comfortably Numb - 2011 Remastered Version            Pink Floyd
                                                     ...        
Kissing a Fool                                    George Michael
(Everything I Do) I Do It For You                    Bryan Adams
Het Dorp - Single Version                          Wim Sonneveld
Stairway to Heaven - 1990 Remaster                  Led Zeppelin
Radar Love - Original Single Edit 1973            Golden Earring
Name: artist, Length: 76, dtype: object

In [62]:
GLOBALCurrentSongList

,artist,danceability,energy,loudness,tempo,valence,popularity,id
title,,,,,,,,
Mistakes,Arnautica,0.489,0.506,-15.873,128.056,0.462,35,5sv3KN8tAtpzNgDsNLPpJ4
Avond,Boudewijn de Groot,0.456,0.370,-15.240,99.455,0.186,56,3n3F07lHLyRKwqg4q64eYA
De Boer Is De Keerl,Normaal,0.519,0.400,-13.266,84.660,0.634,43,02gpJbYl5GhAlYDtLJmls5
Wish You Were Here,Pink Floyd,0.499,0.318,-13.776,121.499,0.282,56,1HzDhHApjdjXPLHF6GGYhu
Comfortably Numb - 2011 Remastered Version,Pink Floyd,0.472,0.366,-12.595,127.167,0.171,67,7Fg4jpwpkdkGCvq1rrXnvx
...,...,...,...,...,...,...,...,...
Kissing a Fool,George Michael,0.394,0.274,-14.314,116.801,0.166,52,4ZCaO5I6XdIHb0X2UwNDtZ
(Everything I Do) I Do It For You,Bryan Adams,0.520,0.363,-12.655,131.224,0.273,18,76qB2ZEZlEJAMqMqUjKusp
Het Dorp - Single Version,Wim Sonneveld,0.382,0.274,-16.409,114.752,0.432,7,29r02BhNetddIKu8xdRigJ
